In [1]:
import numpy as np

from src.focus_lens_quadric_primitive import FocusLensQuadricPrimitive
from src.focus_doe_quadric_primitive import FocusDOEQuadricPrimitive
import src.utils_energy_distribution as en_util

import src.utils_display as disp_util
import src.utils_files as files_util

In [2]:
batch_folder_name = "quad_batches"

save_quads_file_name = 'data/30k16x16.ebv'
load_quads_file_name = 'data/30k16x16.ebv'

load_surface_file_name = 'data/16x16_surf.ebv'

In [3]:
import shutil
import os

# # # Unite quad batches
files_util.unite_quads(save_quads_file_name, folder_name=batch_folder_name)
# # # delete folder
# if os.path.isdir(batch_folder_name):
#     shutil.rmtree(batch_folder_name)

Total number of samples = 30208


Load DATA

In [ ]:
quadric_surface = FocusLensQuadricPrimitive.load(load_surface_file_name)
# quadric_surface = FocusDOEQuadricPrimitive.load(load_surface_file_name)

params, quadrics, req_distrs, rays, design_logs = files_util.load_quads(load_quads_file_name)

num_samples = quadrics.shape[0]

In [ ]:
tracing_results, _ = quadric_surface.trace_rays_batch(quadrics, rays)
rrmse = en_util.masked_rrmse_loss_batch(req_distrs, tracing_results)

print(f'RRMSE: Mean = {rrmse.mean()}; Max = {rrmse.max()}; Min = {rrmse.min()}')

In [ ]:
mean_quads = np.zeros((num_samples, 1))
for i in range(num_samples):
    good_vals = np.isfinite(quadrics[i,:])
    mean_quads[i] = quadrics[i][good_vals].mean()
mid_quad = mean_quads.mean()

disp = ((mean_quads-mid_quad)**2).sum()/num_samples

print(f'Mean quad value = {mid_quad}. Mean quad dispersion = {disp}')

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

ind = disp_util.display_random_pairs([req_distrs], [tracing_results], titles=['Required image','Traced Quadrics'])
disp_util.display_figs([quadrics[ind]], titles=['Calculated Quadrics'])

disp_util.display_2D_func(design_logs['loss'][ind,:],title='Loss per iters')